<a href="https://colab.research.google.com/github/ilovely11/ly11/blob/master/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Housing in California**

The point of this notebook is to create a Linear Regression algorithm from scratch. In this notebook, we used a guide from the machine learning course taught by Guowei Wei at MSU, but our derivation of backpropagating is a little more involved mathematically. 



# About the Dataset

This dataset is from 1990 California Census data, which originally appeared in it's current form in a 1997 paper titled "Sparse Spatial Autoregressions" by R. Kelley Pace and Ronald Barry. The variables in this dataset are longitude, latitutde, median housing price, total rooms, total bedrooms, population per block, household income (in units of 10,000 USD), ocean proximity, and median house value.

#Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import StandardScaler

#Putting Our Data Into a Pandas Dataframe

In [ ]:
train_data = pd.read_csv('/content/sample_data/california_housing_train.csv')

X_train_dataframe = train_data.drop(columns = ['longitude', 'latitude' ,'median_house_value'])
X_train = X_train_dataframe.values

y_train_dataframe = train_data['median_house_value']
y_train = y_train_dataframe.values

test_data = pd.read_csv('/content/sample_data/california_housing_test.csv')

X_test_dataframe = test_data.drop(columns = ['longitude', 'latitude' ,'median_house_value'])
X_test = X_test_dataframe.values

y_test_dataframe = test_data['median_house_value']
y_test = y_test_dataframe.values

In [ ]:
scaler = StandardScaler() # call an object function
scaler.fit(X_train)   # calculate mean
X_train_norm = scaler.transform(X_train)  # apply normalization on X_train
X_test_norm = scaler.transform(X_test)    # apply normalization on X_test

In [ ]:
print(X_train_norm.shape)
print(X_test_norm.shape)
print(y_train.shape)
print(y_test.shape)

(17000, 6)
(3000, 6)
(17000,)
(3000,)
[ 1.36169494  2.29660752 -0.88246225 ...  0.01529238  0.01299867
 -0.377848  ]


# The Theory of Regression With Regularization

We'll start by implementing linear regression with a regularization. In particular, we'll implement a ridge regression since LASSO will take a very similar approach.

We also choose a method involving regularization since we can't solve for the optimal weights using analytical methods. Instead, we'll have to implement numerical methods using gradient descent, which fits more with modern methods used for deep learning.

Since this is a linear regression, we assume that our data can be reasonably modeled by a straight line in some multidimensional space, let's say
$$\hat{y} = XW + b.$$
In this case, $\hat{y}$ is our predicted output, which is the median housing price in a block. The vector $b$ represents the bias. This tell us the difference between the true values and the predicted values in our model. This will be a vector where the value of each entry is the same since it affects our whole dataset globally.

Assume our training set has $N$ datapoints and we have $D$ features. Then let $X$ be the matrix which contains each feature as the column entries and each datapoints is the row. The vector $W$ is the set of weights associated with each feature. Thus, $X$ has shape $(N,D)$ and $W$ has shape $(D,1)$, 

We want the set of weights and bias, $W$ and $b$, that allows us to make the best prediction on our test data. However, how can we measure what is the best fit? We'll check the squared different between our predicted housing prices in a block and our actual housing prices in a block, which we'll call $y$. That is, our loss function will be:
$$L = \frac{\|\hat{y} - y\|_2^2}{2N} = \frac{1}{2N} \sum_{i = 1}^{N} (\hat{y}_i - y_i)^2.$$
To account for overfitting, we'll introduce a regularization parameter, $\alpha$ and let our new loss function be
\begin{align*}
L = \frac{\|\hat{y} - y\|_2^2}{2N} + \frac{\alpha}{2N} ||W||_2^2 &= \frac{1}{2N} \sum_{i = 1}^N (\hat{y}_i - y_i)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} ||w_n||_2^2 \\
&= \frac{1}{2N} \sum_{i = 1}^N (\hat{y}_i - y_i)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2 \\
&= \frac{1}{2N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2.
\end{align*}
where $w_n$ is the $n$th component of the vector $W$ and $x_{i,j}$ is the component in the $i$th row and $j$th column.

We're implementing this with gradient descent. Our unknown parameters we want to find are the matrix $W$ and the value $b$, so we'll find the partial derivatives of our loss function with respect to $b$ and with respect to each component of $W$. Let's start with the $w_m$ where $1 \leq m \leq D$:
\begin{align*}
\frac{\partial L}{\partial w_m} &= \frac{\partial}{\partial w_m}\left(\frac{1}{2N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2\right) \\
&= \frac{\partial}{\partial w_m}\left(\frac{1}{2N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2\right) + \frac{\partial}{\partial w_m}\left(\frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2\right) \\
&= \left(\frac{1}{2N} \sum_{i = 1}^N\frac{\partial}{\partial w_m}\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2\right) + \frac{\partial}{\partial w_m}\left(\frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2\right) \\
&= \frac{1}{N}\sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)\left(\sum_{j = 1}^D\frac{\partial w_j}{\partial w_m}x_{i,j}\right) + \frac{\alpha}{2N} \sum_{n=1}^{N} \frac {\partial}{\partial w_m}(w_1 + \cdots + w_n)^2 \\
&= \frac{1}{N}\sum_{i = 1}^Nx_{i,m}\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right) + \frac{\alpha}{2N} \sum_{n=1}^{N} \frac {\partial}{\partial w_m}(w_1 + \cdots + w_n)^2 \\
&= \sum_{i = 1}^Nx_{i,m}(\hat{y}_i - y_i) + \frac{\alpha}{N} w_m.
\end{align*}

Now, let be the column vector
$$\Delta W = \left(\frac{\partial L}{\partial w_1}, \frac{\partial L}{\partial w_2}, \ldots, \frac{\partial L}{\partial w_D} \right)^T.$$
We can rewrite this as
$$\Delta W = \frac{1}{N}X^T(\hat{y} - y) + \frac{\alpha}{N} W.$$

We'll need to do a similar to thing to find $b$ now. Let's start with some derivatives again: 
\begin{align*}
\frac{\partial L}{\partial b} &= \frac{\partial}{\partial b}\left(\frac{1}{2N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2 + \frac{\alpha}{2N} \sum_{n=1}^{N} (w_1 + \cdots + w_n)^2 \right) \\
&= \frac{\partial}{\partial b}\left(\frac{1}{2N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right)^2 \right) \\
&=  \frac{1}{N} \sum_{i = 1}^N\left(\sum_{j = 1}^D({w_j}x_{i,j}) + b -  y_i\right) \\
&= \frac{1}{N} \sum_{i = 1}^N (\hat{y}_i - y).
\end{align*}


Since we're implementing gradient descent, we need a rate at which our algorithm "learns," which we'll call $\beta$. In more mathematical terms, this is just a scaling factor on our parial derivative that we tweak to find our minimum. We'll update our weights and bias in the following way:
  $$W_{new} = W_{old} - \beta\Delta W,$$
  $$b_{new} = b_{old} - \beta\dfrac{\partial{L}}{\partial{b}}.$$
This is starting to look more like something we can code up now!

# Forward Propagation

This is the easy part! We'll start by calculating what our prediction would be:

In [ ]:
def forward(X, W, b):
    y_hat = np.dot(X, W) + b #adds the scalar b to each entry of our vector
    return y_hat

# Backward Propagation

So, our initial guess probably isn't the minimzing guess, so what we'll do is update our guess for $W$ and $b$ by using gradient descent:

In [ ]:
def gradient_descent(W, b, y, y_hat, X, alpha, beta):
    z = (y_hat-y)/X.shape[0]
    del_b = np.sum(z, axis = 0, keepdims = True)
    del_W = np.dot(X.T, z) + alpha/X.shape[0] * W
    W = W - beta * del_W
    b = b - beta * del_b
    return W, b

# Prediction

Now that we have all the pieces for our algorithm, what's left is to predict after running through enough iterations. We'll take our test data and assume it fits the model. Then we have
$$y_{predicted} = X_{test}W_{optimal} + b_{optimal}.$$

In [ ]:
def predict(W, X_test, b):
    y_pred = np.dot(X_test, W) + b #adds the scalar b to each entry of our vector
    return y_pred

# Checking the Loss

We need to check if our code is actually minimizing our loss, so let's write a function print out the loss:


In [ ]:
def print_loss(y_hat, y):
    return np.linalg.norm(y_hat-y)**2/len(y) 

# Putting it Together

Okay, so now that we have all the required pieces, let's put it all together. As with any numerical algorithm, let's put this through multiple iterations in a loop and then predict:

In [ ]:
alpha = 10
beta = 0.001
W = np.zeros(X_train_norm.shape[1], dtype = 'float')
y_hat = np.zeros(X_train_norm.shape[0], dtype = 'float')
b = 0.0
iterations = 100000 #You'll probably need more iterations to get a good result, but google doesn't really give much run time.
for n in range(iterations):
    y_hat = forward(X_train_norm, W, b)
    W, b = gradient_descent(W, b, y_train, y_hat, X_train_norm, alpha, beta)
    if(n % 100 == 0):
        print(print_loss(y_hat,y_train))
pred = predict(W, X_test_norm, b)
pred = np.matrix.round(pred)

# How accurate is This?

How accurate is this now? Let's check:

In [ ]:
print(y_test)
print(pred)

[344700. 176500. 270500. ...  62000. 162500. 500001.]
[331557. 215508. 276326. ...  87204. 186744. 435574.]


# Conclusion and Discussions

It looks bad, but this is expected since it's linear regression and it doesn't account for any non-linearity. We could probably feature engineer this to make it a little better, but we probably won't get anything that accurate. We've erased any variables about location since it definitely isn't a variable that scales linearly.  However, location is definitely an important part of housing price. This problem probably won't be solved well using regression, so let's try something a little different in our next notebook.